In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import typing as tp
import yaml
import random
import os
import sys
import soundfile as sf
import librosa
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import resnest.torch as resnest_torch

from torchvision import models

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from radam import RAdam


pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [ ]:
# ROOT = Path.cwd().parent
# INPUT_ROOT = ROOT / "input"
INPUT_ROOT = Path("/home/knikaido/work/Cornell-Birdcall-Identification/data")
RAW_DATA = INPUT_ROOT / "birdsong-recognition"
TRAIN_AUDIO_DIR = RAW_DATA / "train_audio"
TRAIN_RESAMPLED_AUDIO_DIRS = [
  INPUT_ROOT / "birdsong-resampled-train-audio-{:0>2}".format(i)  for i in range(5)
]

TEST_AUDIO_DIR = RAW_DATA / "test_audio"

In [ ]:
# train = pd.read_csv(RAW_DATA / "train.csv")
train = pd.read_csv(TRAIN_RESAMPLED_AUDIO_DIRS[0] / "train_mod.csv")
train.head().T


In [ ]:
TRAIN_RESAMPLED_AUDIO_DIRS = [
  INPUT_ROOT / "birdsong-resampled-perc"
]

In [ ]:
tmp_list = []
for audio_d in TRAIN_RESAMPLED_AUDIO_DIRS:
    if not audio_d.exists():
        continue
    for ebird_d in audio_d.iterdir():
        if ebird_d.is_file():
            continue
        for wav_f in ebird_d.iterdir():
            tmp_list.append([ebird_d.name, wav_f.name, wav_f.as_posix()])
            
train_wav_path_exist = pd.DataFrame(
    tmp_list, columns=["ebird_code", "resampled_filename", "file_path"])

del tmp_list

train_all = pd.merge(
    train, train_wav_path_exist, on=["ebird_code", "resampled_filename"], how="inner")

print(train.shape)
print(train_wav_path_exist.shape)
print(train_all.shape)
train_all.head()

In [ ]:
save_dir = '/home/knikaido/work/Cornell-Birdcall-Identification/data/birdsong-resampled-perc-aug/'

for i, file_name in tqdm(enumerate(train_wav_path_exist['file_path'])):
    try:
        y, sr = sf.read(file_name)
        y_harm, y_perc = librosa.effects.hpss(y, margin=(1.0,5.0))
        save_path = save_dir + file_name.split('/', 7)[-1]
        my_makedirs(save_path)
        write(save_path, sr, y_perc)
    except:
        print('err!!!' + file_name)
#     print(save_dir + file_name.split('/', 7)[-1])
#     print(file_name.split('/', 7))
#     if(i==3): break